## **_Análise não Linear Física de uma Estrutura em Forma de Barra_**

**_Disciplina:_** Tópicos em Engenharia de Estruturas VII - Introdução à Análise Não Linear de Estruturas.

### **_Introdução_**

Na engenharia estrutural, um problema é chamado de não linear quando a rigidez da estrutura muda conforme ela se deforma. Em outras palavras, a resistência da estrutura à deformação não é constante e varia com o deslocamento (Fuina, 2025). 

Sendo assim, para o desenvolvimento deste estudo implementamos um modelo numérico para avaliação da resposta estrutural de uma barra submetida a um carregamento distribuído crescente, tendo como objetivo determinar a carga distribuída máxima suportada pela estrutura antes que o deslocamento ultrapasse um valor admissível.

### **_Metodologia_**

A metodologia empregada considera um sistema de três graus de liberdade e utiliza um método iterativo para a resolução das equações de equilíbrio. 

Realizou-se a simulação em Python, utilizando as bibliotecas `numpy` para operações numéricas e `plotly` para visualização dos resultados. 

A formulação do problema baseia-se na discretização de uma barra com três pontos de deslocamento $u_1$, $u_2$ e $u_3$. A matriz de rigidez do sistema é definida em função dos deslocamentos, do módulo de elasticidade inicial $E_0$, da área da seção transversal $S$ e do comprimento da barra $L$, conforme apresentado na função abaixo:

In [ ]:
def calcular_matriz_rigidez(u1, u2, u3, E0, S, L, n):
    factor = (2 * E0 * S) / L
    term = n * 2 / L
    return factor * np.array([
        [1, 0, 0],
        [0, 2 - term * (u3 - u1), -1 + term * (u3 - u2)],
        [0, -1 + term * (u3 - u2), 1 - term * (u3 - u2)]
    ])

O sistema é solucionado de forma *iterativa*, utilizando um critério de convergência implementado na função `resolver_sistema_iterativo`, que recebe o vetor de forças $\mathbf{F}$ e retorna os deslocamentos obtidos após a convergência:

In [ ]:
def resolver_sistema_iterativo(F, tol, nMaxIterations, E0, S, L, n):
    u_sol = np.zeros(3)
    for i in range(nMaxIterations):
        K = calcular_matriz_rigidez(*u_sol, E0, S, L, n)
        u_novo = np.linalg.solve(K, F)
        res = F - K @ u_novo
        if np.linalg.norm(res) < tol and np.linalg.norm(u_sol - u_novo) / (np.linalg.norm(u_novo) + 1e-12) < tol:
            return u_novo, i + 1
        u_sol = u_novo
    print("Aviso: Número máximo de iterações atingido antes da convergência.")
    return u_sol, nMaxIterations

O código principal executa a simulação variando a carga distribuída $q$ até que o deslocamento máximo permitido seja atingido. Os dados são armazenados durante o processo para posterior análise gráfica.

### **_Resultados e Discussões_**

Os resultados obtidos foram plotados utilizando a biblioteca `plotly`, gerando um gráfico que representa a relação entre a carga distribuída $q$ e o deslocamento máximo $u_3$, permitindo identificar o ponto em que a estrutura atinge sua capacidade limite.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=u_max_values,
    y=q_values,
    mode='lines+markers',
    name='Dados',
    line=dict(color='blue'),
    marker=dict(size=8)
))
fig.add_shape(type="line",
              x0=u_max_admissivel, y0=0,
              x1=u_max_admissivel, y1=max(q_values) * 1.1,
              line=dict(color="red", dash="dash"))
fig.update_layout(
    title="Convergência Numérica: Deslocamento u vs. Carga q",
    xaxis_title="Deslocamento máximo u (m)",
    yaxis_title="Carga distribuída q (N/m)",
    template="plotly_white"
)
fig.show()

O gráfico obtido demonstra um crescimento quase linear do deslocamento em relação à carga, seguido por uma região onde a não linearidade se torna mais evidente. Como resultado, a carga distribuída máxima suportada pela estrutura antes de ultrapassar o limite de deslocamento de 1,0 m foi de aproximadamente 1,63 N/m.

<img src="Gráfico Gerado.png" width="500" height="400">

Além disso, observa-se que o critério de convergência foi atendido. No entanto, em alguns pontos, o número máximo de iterações foi alcançado antes da convergência, indicando a necessidade de refinamento na malha ou na solução numérica.

### **_Conclusão_**

O referido projeto demonstrou a aplicabilidade da análise não linear na determinação da capacidade de carga de uma estrutura simples, tal como a de uma barra. O uso de um método iterativo permitiu uma resolução eficiente, fornecendo dados precisos do deslocamento em função da carga aplicada. 

A implementação das bibliotecas em Python demonstrou a eficácia e agilidade dessas ferramentas na resolução numérica e visualização de resultados complexos. Por fim, a abordagem adotada se alinha às metodologias descritas por Figueiredo e Serafini (2020), reforçando a importância das técnicas numéricas na engenharia estrutural.

### **_Referências_**

Beer, F. P.; Johnston, E. R.; Dewolf, J. T.; Mazurek, D. F. *Resistência dos Materiais*. 7. ed. McGraw-Hill, 2017.

Figueiredo, R.; Serafini, M. *Análise Não Linear de Estruturas*. São Paulo: Editora Engenharia, 2020.

Fuina, J. S. *Métodos de Controle de Deformações para Análise Não Linear de Estruturas*. Disponível em: [https://repositorio.ufmg.br/bitstream/1843/LMCA-769HM7/1/153.pdf](https://repositorio.ufmg.br/bitstream/1843/LMCA-769HM7/1/153.pdf). Acesso em: 3 abr. 2025.

Gere, J. M.; Goodno, B. J. *Mecânica dos Materiais*. 9. ed. Cengage Learning, 2020.